# Solver Interface

Each cobrapy solver must expose the following API. The solvers all will have their own distinct LP object types, but each can be manipulated by these functions. This API can be used directly when implementing algorithms efficiently on linear programs because it has 2 primary benefits:

1. Avoid the overhead of creating and destroying LP's for each operation

2. Many solver objects preserve the basis between subsequent LP's, making each subsequent LP solve faster

We will walk though the API with the cglpk solver, which links the cobrapy solver API with [GLPK](http://www.gnu.org/software/glpk/)'s C API.

In [1]:
import cobra.test

model = cobra.test.create_test_model("textbook")
solver = cobra.solvers.cglpk

## Attributes and functions

Each solver has some attributes:

### solver_name

The name of the solver. This is the name which will be used to select the solver in cobrapy functions.

In [2]:
solver.solver_name

'cglpk'

In [3]:
model.optimize(solver="cglpk")

<Solution 0.87 at 0x7fd42ad90c18>

### \_SUPPORTS_MILP

The presence of this attribute tells cobrapy that the solver supports mixed-integer linear programming

In [4]:
solver._SUPPORTS_MILP

True

### solve

Model.optimize is a wrapper for each solver's solve function. It takes in a cobra model and returns a solution

In [5]:
solver.solve(model)

<Solution 0.87 at 0x7fd42ad90908>

### create_problem

This creates the LP object for the solver.

In [6]:
lp = solver.create_problem(model, objective_sense="maximize")
lp

### solve_problem

Solve the LP object and return the solution status

In [7]:
solver.solve_problem(lp)

'optimal'

### format_solution

Extract a cobra.Solution object from a solved LP object

In [8]:
solver.format_solution(lp, model)

<Solution 0.87 at 0x7fd42ad90668>

### get_objective_value

Extract the objective value from a solved LP object

In [9]:
solver.get_objective_value(lp)

0.8739215069684909

### get_status

Get the solution status of a solved LP object

In [10]:
solver.get_status(lp)

'optimal'

### change_variable_objective

change the objective coefficient a reaction at a particular index. This does not change any of the other objectives which have already been set. This example will double and then revert the biomass coefficient.

In [11]:
model.reactions.index("Biomass_Ecoli_core")

12

In [12]:
solver.change_variable_objective(lp, 12, 2)
solver.solve_problem(lp)
solver.get_objective_value(lp)

1.7478430139369818

In [13]:
solver.change_variable_objective(lp, 12, 1)
solver.solve_problem(lp)
solver.get_objective_value(lp)

0.8739215069684909

### change variable_bounds

change the lower and upper bounds of a reaction at a particular index. This example will set the lower bound of the biomass to an infeasible value, then revert it.

In [14]:
solver.change_variable_bounds(lp, 12, 1000, 1000)
solver.solve_problem(lp)

'infeasible'

In [15]:
solver.change_variable_bounds(lp, 12, 0, 1000)
solver.solve_problem(lp)

'optimal'

### change_coefficient

Change a coefficient in the stoichiometric matrix. In this example, we will set the entry for ADP in the ATMP reaction to in infeasible value, then reset it.

In [16]:
model.metabolites.index("atp_c")

16

In [17]:
model.reactions.index("ATPM")

10

In [18]:
solver.change_coefficient(lp, 16, 10, -10)
solver.solve_problem(lp)

'infeasible'

In [19]:
solver.change_coefficient(lp, 16, 10, -1)
solver.solve_problem(lp)

'optimal'

### set_parameter

Set a solver parameter. Each solver will have its own particular set of unique paramters. However, some have unified names. For example, all solvers should accept "tolerance_feasibility."

In [20]:
solver.set_parameter(lp, "tolerance_feasibility", 1e-9)

In [21]:
solver.set_parameter(lp, "objective_sense", "minimize")
solver.solve_problem(lp)
solver.get_objective_value(lp)

0.0

In [22]:
solver.set_parameter(lp, "objective_sense", "maximize")
solver.solve_problem(lp)
solver.get_objective_value(lp)

0.8739215069684912

## Example with FVA

Consider flux variability analysis (FVA), which requires maximizing and minimizing every reaction with the original biomass value fixed at its optimal value. If we used the cobra Model API in a naive implementation, we would do the following:

In [23]:
%%time
# work on a copy of the model so the original is not changed
m = model.copy()

# set the lower bound on the objective to be the optimal value
f = m.optimize().f
for objective_reaction, coefficient in m.objective.items():
    objective_reaction.lower_bound = coefficient * f

# now maximize and minimze every reaction to find its bounds
fva_result = {}
for r in m.reactions:
    m.change_objective(r)
    fva_result[r.id] = {
        "maximum": m.optimize(objective_sense="maximize").f,
        "minimum": m.optimize(objective_sense="minimize").f
    }

CPU times: user 171 ms, sys: 0 ns, total: 171 ms
Wall time: 171 ms


Instead, we could use the solver API to do this more efficiently. This is roughly how cobrapy implementes FVA. It keeps uses the same LP object and repeatedly maximizes and minimizes it. This allows the solver to preserve the basis, and is much faster. The speed increase is even more noticeable the larger the model gets.

In [24]:
%%time
# create the LP object
lp = solver.create_problem(model)

# set the lower bound on the objective to be the optimal value
solver.solve_problem(lp)
f = solver.get_objective_value(lp)
for objective_reaction, coefficient in model.objective.items():
    objective_index = model.reactions.index(objective_reaction)
    # old objective is no longer the objective
    solver.change_variable_objective(lp, objective_index, 0.)
    solver.change_variable_bounds(
        lp, objective_index, f * coefficient,
        objective_reaction.upper_bound)

# now maximize and minimze every reaction to find its bounds
fva_result = {}
for index, r in enumerate(model.reactions):
    solver.change_variable_objective(lp, index, 1.)
    result = {}
    solver.solve_problem(lp, objective_sense="maximize")
    result["maximum"] = solver.get_objective_value(lp)
    solver.solve_problem(lp, objective_sense="minimize")
    result["minimum"] = solver.get_objective_value(lp)
    solver.change_variable_objective(lp, index, 0.)
    fva_result[r.id] = result

CPU times: user 8.28 ms, sys: 25 µs, total: 8.31 ms
Wall time: 8.14 ms
